# Formation PyTorch : les bases pour être autonome 
#### 19 avril 2023de 9h à 17h à l'OMP (salle Coriolis)

# Partie 1
## Manipuler les objets basiques de PyTorch : tenseurs, paramètres, modèles... 

Dans cette première partie, nous allons : 
 * Découvrir et manipuler les éléments de base de PyTorch,
 * Découvrir les bases de l'optimisation avec PyTorch.


In [ ]:
import torch

### Les tenseurs

Le tenseur est l'objet incontournable de PyTorch, l'équivalent du array pour la librairie numpy. La très grande majorité des opérations effectuées avec PyTorch sont effectuées sur des tenseurs. On va voir ci-dessous une liste non exhaustive des opérations possibles sur les tenseurs.

Pour plus d'informations, regardez la documentation : https://pytorch.org/docs/stable/tensors.html?highlight=torch+tensor#torch.Tensor

In [ ]:
X = torch.tensor([
    [2, 1.5, 3],
    [4, 2, 4.5]
])

In [ ]:
X.shape

In [ ]:
X.T

In [ ]:
X.sum()

In [ ]:
X.sum().item()

In [ ]:
torch.sum(X, dim=0)

In [ ]:
X.unsqueeze(1).shape

In [ ]:
X.view(-1, 1).shape

In [ ]:
X.repeat(1, 2)

In [ ]:
torch.zeros((2,2))
torch.ones_like(X)

In [ ]:
torch.randn((2,2))

In [ ]:
X.to('cuda' if torch.cuda.is_available() else 'cpu')

### Paramètres et rétro-propagation

Les paramètres sont des objets dont les valeurs ont vocation à être optimisées, comme par exemple les paramètres d'un réseau de neurones.

https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html?highlight=parameter#torch.nn.parameter.Parameter

In [ ]:
from torch.nn.parameter import Parameter
X = torch.ones(10,1) # un tenseur classique 
W = Parameter(torch.tensor([[2.]]), requires_grad=True) # un paramètre
Y = torch.mm(X, W) # produit matriciel
z = torch.prod(Y) # produit des coordonnées 

`torch.mm(X, W)` effectue le produit matriciel de X et de W: $Y = X W$

Analytiquement, on peut calculer le gradient de $z$ par rapport à $W$ :
    
\begin{align}
    \frac{\partial z}{\partial W} & = \frac{\partial z}{\partial Y} \frac{\partial Y}{\partial W} \\
     & = \big[\frac{\partial z}{\partial Y_1} \ldots \frac{\partial z}{\partial Y_{10}}\big] \big[\frac{\partial Y_1}{\partial Y_W} \ldots \frac{\partial Y_{10}}{\partial W} \big]^T \\ 
     & = \big[\prod_{i=2}^{10} Y_i \ldots \prod_{i=1}^{9} Y_i\big] \big[X_1 \ldots X_{10}\big]^T
\end{align}
    
Finalement, $\large{\frac{\partial z}{\partial W}(W) = 5120}$.

C'est en fait exactement ce que fait PyTorch : à chaque opération réalisée à partir de paramètres, l'attribut `grad_fn` garde en mémoire la fonction qui permet d'évaluer le gradient. Par exemple, `Y.grad_fn` permet de calculer le gradient $\large{\frac{\partial Y}{\partial W}}$.

C'est ce que l'on appelle la diférentiation automatique, qui utilise, comme nous l'avons fait analytiquement, le théorème de la "chain rule", ou théorème de dérivation des fonctions composées en français.

In [ ]:
print(W.grad_fn)
print(Y.grad_fn)
print(z.grad_fn)

La méthode `backward` permet de calculer le gradient de $z$ par rapport à $W$.

In [ ]:
z.backward() 
print(W.grad)

In [ ]:
print(Y.grad)

Par défaut, PyTorch ne calcule pas le gradient de Y car ce n'est pas une feuille de l'arbre de calcul (voir illustratio ci-dessus). Si on le souhaite, il faut utiliser la méthode `retain_grad`.

In [ ]:
X = torch.ones(10,1)
W = Parameter(torch.tensor([[2.]]), requires_grad=True)
Y = torch.mm(X, W)
Y.retain_grad()
z = torch.prod(Y)
z.backward()

print(Y.grad)

On retrouve bien le gradient calculé analytiquement. 
Voyons si on peut également calculer la jacobienne de $Y$ par rapport à $W$.

In [ ]:
X = torch.ones(10,1)
W = Parameter(torch.tensor([[2.]]), requires_grad=True)
Y = torch.mm(X, W)
Y.backward()

Ici, $Y$ n'est pas scalaire. La méthode `backward` ne peut pas implicitement calculer la jacobienne de $Y$ par rapport à $W$. En fait, PyTorch n'est, de base, pas fait pour calculer des quantitités comme des jacobiennes ou des hessiennes. Si vous en avez l'utilité, vous pouvez aller voir ce tutoriel : https://pytorch.org/functorch/stable/notebooks/jacobians_hessians.html

Ci-dessous, on va tout de même voir comment calculer la jacobienne $\large{\frac{\partial Y}{\partial W}}$.

In [ ]:
def f(X, W):
    return torch.mm(X, W)

X = torch.arange(1, 11).reshape((10,1)).float()
W = Parameter(torch.tensor([[2.]]), requires_grad=True)

# Calculer les lignes de la matrice jacobienne
jacobian_rows = ...

jacobian = torch.stack(jacobian_rows)

In [ ]:
jacobian

On retrouve bien ce que l'on avait calculé analytiquement.

### Optimisation des paramètres 

On va voir ci-dessous quels outils utiliser pour l'optimisation des paramètres.

https://pytorch.org/docs/stable/optim.html?highlight=optimizer#torch.optim.Optimizer

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
X = torch.linspace(0, 10, 100).unsqueeze(1)
Y = 2.5 + 3*X + 3*torch.randn(100,1)

fig = plt.figure()
plt.scatter(X, Y)
plt.show()

In [ ]:
import torch.nn.functional as F 

# Complétez la définition de a, le coefficient directeur, et b, l'ordonnée à l'origine
a = ...
b = ...

# Complétez la définition de l'optimiseur : 
# https://pytorch.org/docs/stable/generated/torch.optim.SGD.html?highlight=sgd#torch.optim.SGD
optimizer = torch.optim.SGD(..., lr=1e-2)

for epoch in range(10):
    # Complétez le calcul de y_pred
    y_pred = ...
    mse = F.mse_loss(Y, y_pred)
    mse.backward() # On calcule les gradients de mse par rapport à a et b 
    optimizer.step() # On met à jour les valeurs de a et b 
    optimizer.zero_grad() # On 'remet' les gradients à zéro pour la prochaine epoch 
    print(f'=== Epoch {epoch} ===  ')
    print(f'MSE: {mse.item():.3f}')
    

In [ ]:
with torch.no_grad():
    y_pred = torch.mm(X, a) + b 
    
fig = plt.figure()
plt.scatter(X, Y)
plt.plot(X, y_pred, color='red')
plt.show()

### Les modèles

Les modèles sont les objets de PyTorch qui définissent les réseaux de neurones.

https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module

In [ ]:
class LinearModel(torch.nn.Module):
    def __init__(self, x_dim):
        super(LinearModel, self).__init__()
        self.x_dim = x_dim
        self.linear = torch.nn.Linear(x_dim, 1)
        
    def forward(self, x):
        return self.linear(x)

In [ ]:
linear_model = LinearModel(X.shape[1])
linear_model

In [ ]:
for param in linear_model.named_parameters():
    print(param[0])

In [ ]:
linear_model.linear.weight

In [ ]:
linear_model.linear.weight.shape

In [ ]:
linear_model.linear.bias

In [ ]:
optimizer = torch.optim.SGD(..., lr=1e-2)

for epoch in range(10):
    y_pred = ...
    mse = F.mse_loss(Y, y_pred)
    mse.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f'=== Epoch {epoch} ===  ')
    print(f'MSE: {mse.item():.3f}')

In [ ]:
with torch.no_grad():
    y_pred = ...
    
fig = plt.figure()
plt.scatter(X, Y)
plt.plot(X, y_pred, color='red')
plt.show()

### Data Loaders

Les data loader sont des objets de PyTorch pour charger les données de manière à faciliter l'optimisation par "Batch stochastic gradient descent".

https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader

In [ ]:
X = torch.linspace(0, 10, 100).unsqueeze(1)
Y = 2.5 + 3*X + 3*torch.randn((100,1))

data = torch.utils.data.TensorDataset(X, Y)

In [ ]:
data

In [ ]:
len(data)

In [ ]:
loader = torch.utils.data.DataLoader(data, batch_size=20, shuffle=True)

In [ ]:
len(loader)

In [ ]:
optimizer = torch.optim.SGD(linear_model..., lr=1e-2)

for epoch in range(10):
    for batch, y in loader:
        y_pred = ...
        mse = F.mse_loss(y, y_pred)
        mse.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f'=== Epoch {epoch} ===  ')
    print(f'MSE: {mse.item():.3f}')